In [ ]:
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
import pedpy
import sqlite3
import pandas as pd
from shapely import GeometryCollection, Polygon

In [ ]:
trajectory_file = "./corner_roundrobin_1.sqlite"

In [ ]:
def read_sqlite_file(trajectory_file: str) -> pedpy.TrajectoryData:
    with sqlite3.connect(trajectory_file) as con:
        data = pd.read_sql_query(
            "select frame, id, pos_x as x, pos_y as y, ori_x as ox, ori_y as oy from trajectory_data",
            con,
        )
        fps = float(
            con.cursor()
            .execute("select value from metadata where key = 'fps'")
            .fetchone()[0]
        )
        walkable_area = (
            con.cursor().execute("select wkt from geometry").fetchone()[0]
        )
        return (
            pedpy.TrajectoryData(data=data, frame_rate=fps),
            pedpy.WalkableArea(walkable_area),
        )

In [ ]:
trajectory_data, walkable_area = read_sqlite_file(trajectory_file)
speed = pedpy.compute_individual_speed(traj_data=trajectory_data, frame_step=5)
speed = speed.merge(trajectory_data.data, on=["id", "frame"], how="left")
speed["radius"] = 0.2

In [ ]:
def speed_to_color(value, vmin, vmax, colormap="Jet"):
    """Map a speed value to a color using a colormap."""
    norm_value = (value - vmin) / (vmax - vmin)
    colorscale = px.colors.sequential.__dict__[colormap][::-1]
    index = int(norm_value * (len(colorscale) - 1))
    index = min(max(index, 0), len(colorscale) - 1)
    color = colorscale[index]
    return color

In [ ]:
def get_geometry_traces(area):
    geometry_traces = []
    x, y = area.exterior.xy
    geometry_traces.append(
        go.Scatter(
            x=np.array(x),
            y=np.array(y),
            mode="lines",
            line={"color": "grey"},
            showlegend=False,
            name="Exterior",
            hoverinfo="name",
        )
    )
    for inner in area.interiors:
        xi, yi = zip(*inner.coords[:])
        geometry_traces.append(
            go.Scatter(
                x=np.array(xi),
                y=np.array(yi),
                mode="lines",
                line={"color": "grey"},
                showlegend=False,
                name="Obstacle",
                hoverinfo="name",
            )
        )
    return geometry_traces

In [ ]:
def get_scatter_for_frame(frame_data, min_speed, max_speed):
    """Utilize scatter plots with varying colors for each agent instead of individual shapes.

    This trace is only to incorporate a colorbar in the plot.
    """
    scatter_trace = go.Scatter(
        x=frame_data["x"],
        y=frame_data["y"],
        mode="markers",
        marker=dict(
            size=frame_data["radius"] * 2,
            color=frame_data["speed"],
            colorscale="Jet_r",
            colorbar=dict(title="Speed"),
            cmin=min_speed,
            cmax=max_speed,
        ),
        text=frame_data["speed"],
        showlegend=False,
        hoverinfo="none",
    )

    return [scatter_trace]

In [ ]:
def get_shapes_for_frame(frame_data, min_speed, max_speed):
    def create_shape(row):
        hover_trace = go.Scatter(
            x=[row["x"]],
            y=[row["y"]],
            text=[f"ID: {row['id']}, Pos({row['x']:.2f},{row['y']:.2f})"],
            mode="markers",
            marker=dict(size=1, opacity=1),
            hoverinfo="text",
            showlegend=False,
        )
        dummy_trace = go.Scatter(
            x=[row["x"]],
            y=[row["y"]],
            mode="markers",
            marker=dict(size=1, opacity=0),
            hoverinfo="none",
            showlegend=False,
        )
        if row["speed"] == -1000:  # Check for dummy speed
            return (
                go.layout.Shape(
                    type="circle",
                    xref="x",
                    yref="y",
                    x0=row["x"] - row["radius"],
                    y0=row["y"] - row["radius"],
                    x1=row["x"] + row["radius"],
                    y1=row["y"] + row["radius"],
                    line=dict(width=0),
                    fillcolor="rgba(255,255,255,0)",  # Transparent fill
                ),
                dummy_trace,
            )
        color = speed_to_color(row["speed"], min_speed, max_speed)
        return (
            go.layout.Shape(
                type="circle",
                xref="x",
                yref="y",
                x0=row["x"] - row["radius"],
                y0=row["y"] - row["radius"],
                x1=row["x"] + row["radius"],
                y1=row["y"] + row["radius"],
                line_color=color,
                fillcolor=color,
            ),
            hover_trace,
        )

    results = frame_data.apply(create_shape, axis=1).tolist()
    shapes = [res[0] for res in results]
    hover_traces = [res[1] for res in results]
    return shapes, hover_traces

In [ ]:
def create_fig(
    initial_agent_count,
    initial_shapes,
    initial_hover_trace,
    initial_scatter_trace,
    geometry_traces,
    hover_traces,
    frames,
    steps,
    area_bounds,
    width=800,
    height=800,
):
    """Creates a Plotly figure with animation capabilities.

    Returns:
        go.Figure: A Plotly figure with animation capabilities.
    """

    minx, miny, maxx, maxy = area_bounds
    title = f"<b>Number of Agents: {initial_agent_count}</b>"
    fig = go.Figure(
        data=geometry_traces
        + initial_scatter_trace
        + hover_traces
        + initial_hover_trace,
        frames=frames,
        layout=go.Layout(shapes=initial_shapes, title=title, title_x=0.5),
    )
    fig.update_layout(
        updatemenus=[_get_animation_controls()],
        sliders=[_get_slider_controls(steps)],
        autosize=False,
        width=width,
        height=height,
        xaxis=dict(range=[minx - 0.5, maxx + 0.5]),
        yaxis=dict(
            scaleanchor="x", scaleratio=1, range=[miny - 0.5, maxy + 0.5]
        ),
    )

    return fig


def _get_animation_controls():
    """Returns the animation control buttons for the figure."""
    return {
        "buttons": [
            {
                "args": [
                    None,
                    {
                        "frame": {"duration": 100, "redraw": True},
                        "fromcurrent": True,
                    },
                ],
                "label": "Play",
                "method": "animate",
            },
            {
                "args": [
                    [None],
                    {
                        "frame": {"duration": 0, "redraw": True},
                        "mode": "immediate",
                        "transition": {"duration": 0},
                    },
                ],
                "label": "Pause",
                "method": "animate",
            },
        ],
        "direction": "left",
        "pad": {"r": 10, "t": 87},
        "showactive": False,
        "type": "buttons",
        "x": 0.1,
        "xanchor": "right",
        "y": 0,
        "yanchor": "top",
    }


def _get_slider_controls(steps):
    """Returns the slider controls for the figure."""
    return {
        "active": 0,
        "yanchor": "top",
        "xanchor": "left",
        "currentvalue": {
            "font": {"size": 20},
            "prefix": "Frame:",
            "visible": True,
            "xanchor": "right",
        },
        "transition": {"duration": 100, "easing": "cubic-in-out"},
        "pad": {"b": 10, "t": 50},
        "len": 0.9,
        "x": 0.1,
        "y": 0,
        "steps": steps,
    }

In [ ]:
def animate(
    data: pedpy.TrajectoryData, area: pedpy.WalkableArea, *, every_nth_frame=50
):
    data_df = pedpy.compute_individual_speed(traj_data=data, frame_step=5)
    data_df = data_df.merge(data.data, on=["id", "frame"], how="left")
    data_df["radius"] = 0.2
    min_speed = data_df["speed"].min()
    max_speed = data_df["speed"].max()
    max_agents = data_df.groupby("frame").size().max()
    frames = []
    steps = []
    unique_frames = data_df["frame"].unique()
    selected_frames = unique_frames[::every_nth_frame]
    geometry_traces = get_geometry_traces(area.polygon)
    initial_frame_data = data_df[data_df["frame"] == data_df["frame"].min()]
    initial_agent_count = len(initial_frame_data)
    initial_shapes, initial_hover_trace = get_shapes_for_frame(
        initial_frame_data, min_speed, max_speed
    )
    initial_scatter_trace = get_scatter_for_frame(
        initial_frame_data, min_speed, max_speed
    )

    for frame_num in selected_frames[1:]:
        frame_data, agent_count = _get_processed_frame_data(
            data_df, frame_num, max_agents
        )
        shapes, hover_traces = get_shapes_for_frame(
            frame_data, min_speed, max_speed
        )
        frame = go.Frame(
            data=geometry_traces + hover_traces,
            name=str(frame_num),
            layout=go.Layout(
                shapes=shapes,
                title=f"<b>Number of Agents: {agent_count}</b>",
                title_x=0.5,
            ),
        )
        frames.append(frame)

        step = {
            "args": [
                [str(frame_num)],
                {
                    "frame": {"duration": 100, "redraw": True},
                    "mode": "immediate",
                    "transition": {"duration": 500},
                },
            ],
            "label": str(frame_num),
            "method": "animate",
        }
        steps.append(step)

    return create_fig(
        initial_agent_count,
        initial_shapes,
        initial_hover_trace,
        initial_scatter_trace,
        geometry_traces,
        hover_traces,
        frames,
        steps,
        area.bounds,
        width=800,
        height=800,
    )


def _get_processed_frame_data(data_df, frame_num, max_agents):
    """Process frame data and ensure it matches the maximum agent count."""
    DUMMY_SPEED = -1000
    frame_data = data_df[data_df["frame"] == frame_num]
    agent_count = len(frame_data)
    dummy_agent_data = {"x": 0, "y": 0, "radius": 0, "speed": DUMMY_SPEED}
    while len(frame_data) < max_agents:
        dummy_df = pd.DataFrame([dummy_agent_data])
        frame_data = pd.concat([frame_data, dummy_df], ignore_index=True)
    return frame_data, agent_count

In [ ]:
trajectory_data, walkable_area = read_sqlite_file(trajectory_file)
animate(trajectory_data, walkable_area)

In [ ]:
trajectory_data